<a href="https://colab.research.google.com/github/theokleiao/AbusiveWords_detector/blob/master/NLTK_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
import string
import nltk
nltk.download()
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.externals import joblib

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [

In [0]:
train = pd.read_csv("https://raw.githubusercontent.com/theokleiao/AbusiveWords_detector/master/train_E6oV3lV.csv", encoding='utf8')

In [0]:
train.head()
trainTweet = train.tweet
trainTweet.head()

0     @user when a father is dysfunctional and is s...
1    @user @user thanks for #lyft credit i can't us...
2                                  bihday your majesty
3    #model   i love u take with u all the time in ...
4               factsguide: society now    #motivation
Name: tweet, dtype: object

In [0]:
train['label'] = train['label'].astype('category')

In [0]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
id       31962 non-null int64
label    31962 non-null category
tweet    31962 non-null object
dtypes: category(1), int64(1), object(1)
memory usage: 530.8+ KB


In [0]:
from google.colab import files
uploaded = files.upload()

Saving Train.txt to Train.txt


In [0]:
filename = 'Train.txt'
file = open(filename, 'rt')
text = file.read()
file.close()
words = text.split()
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in words]

In [0]:
#words = trainTweet.split()
words = [word.lower() for word in words]
print(words[:100])

['@user', 'when', 'a', 'father', 'is', 'dysfunctional', 'and', 'is', 'so', 'selfish', 'he', 'drags', 'his', 'kids', 'into', 'his', 'dysfunction.', '#run', '@user', '@user', 'thanks', 'for', '#lyft', 'credit', 'i', "can't", 'use', 'cause', 'they', "don't", 'offer', 'wheelchair', 'vans', 'in', 'pdx.', '#disapointed', '#getthanked', 'bihday', 'your', 'majesty', '#model', 'i', 'love', 'u', 'take', 'with', 'u', 'all', 'the', 'time', 'in', 'urð\x9f\x93±!!!', 'ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91', 'ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦', 'factsguide:', 'society', 'now', '#motivation', '[2/2]', 'huge', 'fan', 'fare', 'and', 'big', 'talking', 'before', 'they', 'leave.', 'chaos', 'and', 'pay', 'disputes', 'when', 'they', 'get', 'there.', '#allshowandnogo', '@user', 'camping', 'tomorrow', '@user', '@user', '@user', '@user', '@user', '@user', '@user', 'dannyâ\x80¦', 'the', 'next', 'school', 'year', 'is', 'the', 'year', 'for', 'exams.ð\x9f\x98¯', "can't", 'think', 'about']


In [0]:
from nltk import sent_tokenize
nltk.download('punkt')
sentences = sent_tokenize(str(words))
print(sentences[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['@user', 'when', 'a', 'father', 'is', 'dysfunctional', 'and', 'is', 'so', 'selfish', 'he', 'drags', 'his', 'kids', 'into', 'his', 'dysfunction.


In [0]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(str(words))
words = [word for word in tokens if word.isalpha()]
print(words[:100])
print(tokens[:100])

['user', 'run', 'user', 'user', 'lyft', 'ca', 'do', 'disapointed', 'getthanked', 'model', 'motivation', 'allshowandnogo', 'user', 'user', 'user', 'user', 'user', 'user', 'user', 'user', 'ca', 'school', 'exams', 'hate', 'imagine', 'actorslife', 'revolutionschool', 'girl', 'allin', 'cavs', 'champions', 'cleveland', 'clevelandcavaliers', 'user', 'user', 'i', 'it', 'ireland', 'mom', 'blog', 'silver', 'gold', 'forex', 'orlando', 'standwithorlando', 'pulseshooting', 'orlandoshooting', 'biggerproblems', 'selfish', 'heabreaking', 'values', 'love', 'gettingfed', 'user', 'cnn', 'michigan', 'wall', 'tcot', 'australia', 'opkillingbay', 'seashepherd', 'helpcovedolphins', 'thecove', 'helpcovedolphins', 'junior', 'junior', 'yugyoem', 'omg', 'thankful', 'positive', 'friday', 'user', 'cookies', 'badday', 'coneofshame', 'cats', 'pissed', 'funny', 'laughs', 'wine', 'who', 'it', 'weekend', 'amp', 'user', 'user', 'user', 'tgif', 'ff', 'gamedev', 'indiedev', 'indiegamedev', 'squad', 'user', 'user', 'user', 

In [0]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
print(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [0]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
stemmed = [porter.stem(word) for word in tokens]
print(stemmed[:100])

['[', "'", '@', 'user', "'", ',', "'when", "'", ',', "'a", "'", ',', "'father", "'", ',', "'i", "'", ',', "'dysfunct", "'", ',', "'and", "'", ',', "'i", "'", ',', "'so", "'", ',', "'selfish", "'", ',', "'he", "'", ',', "'drag", "'", ',', "'hi", "'", ',', "'kid", "'", ',', "'into", "'", ',', "'hi", "'", ',', "'dysfunct", '.', "'", ',', "'", '#', 'run', "'", ',', "'", '@', 'user', "'", ',', "'", '@', 'user', "'", ',', "'thank", "'", ',', "'for", "'", ',', "'", '#', 'lyft', "'", ',', "'credit", "'", ',', "'i", "'", ',', '``', 'ca', "n't", "''", ',', "'use", "'", ',', "'caus", "'", ',', "'they", "'"]


In [0]:
data = pd.read_csv('clean_data.csv')
texts = words['text'].astype(str)
y = data['is_offensive']

TypeError: ignored

In [0]:
vectorizer = CountVectorizer(stop_words='english', min_df=0.0001)
X = vectorizer.fit_transform(stemmed)

In [0]:
model - LinearSVC(class_weight='balanced', dual=False, tol=1e-2, max_iter=1e5)
cclf = CalibratedClassifierCV(base_estimator=model)
cclf.fit(X,y)

In [0]:
joblib.dump(vectorizer, 'vectorizer.joblib')
joblib.dump(cclf, 'model.joblib')